In [3]:
pip install fosforml

     |████████████████████████████████| 51kB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 13.1MB/s eta 0:00:01��██████████████████████████▌   | 1.7MB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 111.3MB/s eta 0:00:01B/s eta 0:00:01               | 2.8MB 111.3MB/s eta 0:00:01 0:00:010:00:01█████████████████▌     | 9.0MB 111.3MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 98.7MB/s eta 0:00:01
     |████████████████████████████████| 133kB 110.0MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 75kB/s /s eta 0:00:01.3MB/s eta 0:00:03 |▊                               | 4.5MB 75.3MB/s eta 0:00:03                     | 12.1MB 75.3MB/s eta 0:00:03 0:00:03:00:03               | 21.1MB 75.3MB/s eta 0:00:03███▏                          | 32.2MB 75.3MB/s eta 0:00:03    |█████▌                          | 34.5MB 75.3MB/s eta 0:00:03██████▍                         | 39.8MB 75.3MB/s eta 0:00:03███████                         | 4

In [1]:
pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from fosforml.model_manager.snowflakesession import get_session

In [4]:
my_session = get_session()

In [5]:
table_name = 'Fact_Customer_Orders'
sf_df = my_session.sql("select * from {}".format(table_name)) 
 
df = sf_df.to_pandas() 
print(df.shape)

(225563, 4)


In [6]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [9]:
my_session.connection.database 

'MFG_DEMAND_ANALYTICS'

In [10]:
my_session.connection.schema

'GOLD'

In [11]:
type(sf_df)

snowflake.snowpark.dataframe.DataFrame

In [12]:
type(df)

pandas.core.frame.DataFrame

In [13]:
df.head(5)

,ORDER_ID,PRODUCT_ID,ORDER_QTY,ORDER_DATE
0,Order_000006,Product_001,850,2019-01-02
1,Order_000001,Product_004,700,2019-01-02
2,Order_000003,Product_004,900,2019-01-02
3,Order_000008,Product_004,550,2019-01-02
4,Order_000012,Product_004,300,2019-01-02


In [14]:
df.describe()

,ORDER_QTY
count,225563.000000
mean,525.099418
std,288.681633
min,50.000000
25%,250.000000
50%,550.000000
75%,800.000000
max,1000.000000


In [15]:
df.isnull().sum()

ORDER_ID      0
PRODUCT_ID    0
ORDER_QTY     0
ORDER_DATE    0
dtype: int64

In [16]:
df.duplicated().sum()

0

In [17]:
df.corr(numeric_only=True)

,ORDER_QTY
ORDER_QTY,1.0


In [18]:
import matplotlib.pyplot as plt 

In [1]:
pip install statsmodels

     |████████████████████████████████| 10.8MB 6.8MB/s eta 0:00:018MB/s eta 0:00:01
     |████████████████████████████████| 235kB 106.9MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 


In [5]:
table_name = '"Fact_Customer_Orders"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

SnowparkSQLException: (1304): 01b74eaf-0003-0cb3-0000-31ef0011b062: 002003 (42S02): SQL compilation error:
Object '"Fact_Customer_Orders"' does not exist or not authorized.

In [6]:
table_name = 'Fact_Customer_Orders'
sf_df = my_session.sql("select * from {}".format(table_name)) 
 
df = sf_df.to_pandas() 

In [7]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [8]:
data = df[['PRODUCT_ID','ORDER_QTY', 'ORDER_DATE']]

In [1]:
#data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_QTY   225563 non-null  int16         
 2   ORDER_DATE  225563 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB


In [11]:
data = data.set_index('ORDER_DATE')

In [12]:
data.sort_index(inplace=True)

In [13]:
data

,PRODUCT_ID,ORDER_QTY
ORDER_DATE,,
2019-01-02,Product_001,850
2019-01-02,Product_004,700
2019-01-02,Product_004,900
2019-01-02,Product_004,550
2019-01-02,Product_004,300
...,...,...
2024-08-31,Product_098,200
2024-08-31,Product_100,800
2024-08-31,Product_100,550


In [14]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [15]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'test_forecast_order_qty','rse', 'rmse','r2_score'])

In [16]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'forecast_order_qty'])

In [2]:
for PRODUCT_ID, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    
    date_list = test_df.index.to_list()
    
    new_row = {
        'PRODUCT_ID': PRODUCT_ID,
        'ORDER_DATE': date_list,
        'test_forecast_order_qty': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'PRODUCT_ID': PRODUCT_ID,
        'ORDER_DATE': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

NameError: name 'grouped_data' is not defined

In [22]:
test_results

,PRODUCT_ID,ORDER_DATE,test_forecast_order_qty,rse,rmse,r2_score
0,Product_001,13530,537.353131,0.0,290.185585,-0.000545
1,Product_001,13531,549.804686,0.0,290.185585,-0.000545
2,Product_001,13532,538.368357,0.0,290.185585,-0.000545
3,Product_001,13533,553.529134,0.0,290.185585,-0.000545
4,Product_001,13534,525.990260,0.0,290.185585,-0.000545
...,...,...,...,...,...,...
51891,Product_100,3886,476.050933,0.0,295.283780,-0.021838
51892,Product_100,3887,522.781958,0.0,295.283780,-0.021838
51893,Product_100,3888,475.125436,0.0,295.283780,-0.021838
51894,Product_100,3889,498.165467,0.0,295.283780,-0.021838


In [23]:
forecast_results

,PRODUCT_ID,ORDER_DATE,forecast_order_qty
0,Product_001,13530,537.353131
1,Product_001,13531,549.804686
2,Product_001,13532,538.368357
3,Product_001,13533,553.529134
4,Product_001,13534,525.990260
...,...,...,...
1095,Product_100,3132,480.522295
1096,Product_100,3133,503.618313
1097,Product_100,3134,506.115285
1098,Product_100,3135,519.150990
